In [1]:
from datetime import datetime, timedelta
import json
from pymlb_statsapi import api as mlb


In [ ]:
def jprint(obj: dict | list) -> None:
    print(json.dumps(obj, indent=2))


def getTeamId(teamName: str, season: int = None) -> int:
    seasonYear = season or datetime.now().year
    response = api.Team.teams(sportId=1, season=seasonYear)
    teamsData = response.json().get("teams", [])
    teamLookup = []
    for team in teamsData:
        namesList = [
            team.get("name").lower(),
            team.get("teamName").lower(),
            team.get("abbreviation").lower(),
            team.get("shortName").lower(),
            team.get("locationName").lower(),
        ]
        teamLookup.append(
            {
                "id": team.get("id"),
                "name": team.get("name"),
                "namesList": namesList,
            }
        )
    for team in teamLookup:
        if teamName.lower() in team.get("namesList", []):
            return team.get("id")
    return None


def getTeamRoster(teamName: str, season: int = None) -> list:
    teamId = getTeamId(teamName, season)
    if not teamId:
        return []
    year = season or datetime.now().year
    response = api.Team.roster(teamId=teamId, season=year, rosterType="active").json()
    rosterData = response.get("roster", [])
    return rosterData


In [ ]:
jprint(getTeamRoster("Braves"))


In [ ]:
def printTeamRoster(teamName: str, season: int = None) -> None:
    roster = getTeamRoster(teamName, season)
    teamRoster = [
        {
            "name": player.get("person", {}).get("fullName"),
            "jerseyNumber": player.get("jerseyNumber"),
            "position": player.get("position", {}).get("abbreviation"),
            "positionGroup": (
                "Pitchers"
                if player.get("position", {}).get("abbreviation") == "P"
                else (
                    "Infielders"
                    if player.get("position", {}).get("abbreviation")
                    in ["C", "1B", "2B", "SS", "3B"]
                    else (
                        "OutFielders"
                        if player.get("position", {}).get("abbreviation")
                        in ["LF", "CF", "RF"]
                        else "Other"
                    )
                )
            ),
        }
        for player in roster
    ]
    teamRoster = sorted(
        teamRoster,
        key=lambda x: (
            x["positionGroup"],
            int(x["jerseyNumber"]) if x["jerseyNumber"] else 999,
            x["name"],
        ),
    )

    maxNumberLength = len(
        max([player.get("jerseyNumber") for player in teamRoster], key=len)
    )
    maxNameLength = len(max([player.get("name") for player in teamRoster], key=len))
    maxPositionLength = len(
        max([player.get("position") for player in teamRoster], key=len)
    )

    header = f" # | {'Player':<{maxNameLength}} | {'Pos':<{maxPositionLength}}"

    print(header)
    print(f"---+{'-' * (maxNameLength + 2)}+{'-' * (maxPositionLength + 2)}")
    for player in teamRoster:
        number = player.get("jerseyNumber") or "--"
        name = player.get("name")
        position = player.get("position")
        print(
            f"{number:>{maxNumberLength}} | {name:<{maxNameLength}} | {position:<{maxPositionLength}}"
        )


In [ ]:
printTeamRoster("Braves")


#### Examples That Return Formatted/Print Results...


In [ ]:
def printTeamTotalSeasonWins(teamName: str, season: int = None) -> None:
    teamId = getTeamId(teamName)
    year = season or datetime.now().year
    isCurrSeason = year == datetime.now().year

    response = api.Schedule.schedule(
        sportId=1, teamId=teamId, startDate=f"{year}-01-01", endDate=f"{year}-12-31"
    ).json()

    gameDates = response.get("dates", [])

    totalWins = 0
    for games in gameDates:
        for game in games.get("games", []):
            teams = game.get("teams", {})
            homeTeamResult = {
                "id": teams.get("home", {}).get("team", {}).get("id"),
                "isWinner": teams.get("home", {}).get("isWinner", False),
            }
            awayTeamResult = {
                "id": teams.get("away", {}).get("team", {}).get("id"),
                "isWinner": teams.get("away", {}).get("isWinner", False),
            }
            winningTeamId = (
                homeTeamResult["id"]
                if homeTeamResult["isWinner"]
                else awayTeamResult["id"] if awayTeamResult["isWinner"] else None
            )

            if winningTeamId == teamId:
                totalWins += 1

    print(
        f"The {teamName}{' have won ' if isCurrSeason else ' won '}{totalWins} games {'this season' if isCurrSeason else f'in {year}'}."
    )


In [ ]:
printTeamTotalSeasonWins("Braves", 2021)
printTeamTotalSeasonWins("Braves", 2023)
printTeamTotalSeasonWins("Braves", 2025)
printTeamTotalSeasonWins("Braves")


#### SOME MORE COOL EXAMPLES TO TRY AND CONVERT TO USING THIS NEW API...


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )

# # Example: Print the Braves 40-man Roster on opening day of the 2025 season
# print(
#     "Braves 40-man roster on opening day of the 2025 season:\n%s"
#     % statsapi.roster(
#         144,
#         "40Man",
#         date=statsapi.get("season", {"seasonId": 2025, "sportId": 1})["seasons"][0][
#             "regularSeasonStartDate"
#         ],
#     )
# )

# # Example: Print boxscore and linescore from Braves most recent game (which may be in progress or may not have started yet based on MLB response to 'last game' request)
# most_recent_game_id = statsapi.last_game(144)
# print(
#     statsapi.boxscore(most_recent_game_id),
#     statsapi.linescore(most_recent_game_id),
#     sep="\n\n",
# )

# # Other Examples:
# # - find team with longest name
# longest_team_name = max(
#     [
#         x["name"]
#         for x in statsapi.get(
#             "teams", {"sportIds": 1, "activeStatus": "Yes", "fields": "teams,name"}
#         )["teams"]
#     ],
#     key=len,
# )
# print(
#     "The team with the longest name is %s, at %s characters."
#     % (longest_team_name, len(longest_team_name))
# )

# # - print standings from July 4, 2025
# print(statsapi.standings(date="07/04/2025"))

# #  - print top 5 team leaders in walks for 2025 Braves
# print(statsapi.team_leaders(144, "walks", limit=5, season=2025))

# # - print top 10 all time career leaders in doubles
# print(
#     statsapi.league_leaders("doubles", statGroup="hitting", statType="career", limit=10)
# )

# # - print Ronald Acuña Jr.'s career hitting stats
# print(
#     statsapi.player_stats(
#         statsapi.lookup_player("Ronald Acuña Jr.")[0]["id"],
#         "hitting",
#         "career",
#     )
# )

# # - print list of scoring plays from 6/23/2024 Braves @ Yankees
# print(statsapi.game_scoring_plays(745727))
